In [1]:
#export
import k1lib as _k1lib
import matplotlib.pyplot as _plt

In [2]:
#export
class SliceablePlot:
    def __init__(self, plotF, _slice=slice(None, None, None), alphaSlice=None, attrs=[], docs=""):
        self.plotF = plotF
        self.slice = _slice
        self.alphaSlice = alphaSlice # slice using list of strings or string
        self.attrs = attrs
        self.docs = docs
    def __getattr__(self, attr):
        return SliceablePlot(self.plotF, self.slice, self.alphaSlice, self.attrs + [attr], self.docs)
    def __getitem__(self, _slice):
        if type(_slice) == slice:
            return SliceablePlot(self.plotF, _slice, self.alphaSlice, self.attrs, self.docs)
        else: return SliceablePlot(self.plotF, self.slice, _slice, self.attrs, self.docs)
    def __repr__(self):
        if self.alphaSlice == None:
            if len(self.attrs) > 0: self.plotF(self.slice, attrs=self.attrs)
            else: self.plotF(self.slice)
        else:
            if len(self.attrs) > 0:
                self.plotF(self.slice, self.alphaSlice, attrs=self.attrs)
            else: self.plotF(self.slice, self.alphaSlice)
        return f"""Sliceable plot. Can...
- p[a:b]: to focus on a specific range of the plot{self.docs}"""

In [3]:
!../export.py viz

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> build/lib/k1lib
copying k1lib/viz.py -> build/l